In [2]:
import matplotlib.pyplot as plt
from pprint import pprint
import sys
sys.path.insert(0, '..')
from fashion import utils

# show all results of the notebook not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# get (my private) API key
api_key = utils.get_api_key()

# load the shops data
shops = utils.load_shops('../data/20200120_filiali.csv')
shops.head()

#oop

,Store_Key,Franchise,Zip_Code,City
0,2d459c389e1a0561a2953b95cfdc0caa,I,95024,ACIREALE
1,c93364d5937332fea742adf1bb0fd67e,,37010,AFFI
2,8316b325ea6fd3bdeb794ffb649eb556,,80021,AFRAGOLA
3,c359def3032833a2ebbc12c8948a18f3,,51031,AGLIANA
4,293e777d87dea6fe28930c91bb2f99fa,I,86081,AGNONE


In [3]:
sales = utils.load_sales('../data/20200120_sales17.csv')
sales.head()

,Store_Key,Recipt_Key,Date,Hour,Barcode,No_sold,Net_Income
0,2d459c389e1a0561a2953b95cfdc0caa,160217,20161210,1318,2088988001033,1,19.95
1,2d459c389e1a0561a2953b95cfdc0caa,160253,20161210,1634,2088988001033,1,19.95
2,2d459c389e1a0561a2953b95cfdc0caa,160319,20161210,1911,2088988001057,1,19.95
3,2d459c389e1a0561a2953b95cfdc0caa,160330,20161210,1931,2088988001064,1,19.95
4,2d459c389e1a0561a2953b95cfdc0caa,160499,20161211,1747,2088986001042,1,19.95


In [4]:
items = utils.load_Product('../data/20200120_barcode.csv')
items.head()

c:\users\odhra\appdata\local\conda\conda\envs\fashion\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,Barcode,Product_identifier,Color_code,Color_description,Size_code,Gender,Item,Item_description,Product_category_code,season_code,...,size_ID,Original_list_price,COG,Recieved_items,Introduction_period,rotation,price_range,fashion_content,seasonality,innovation_content
0,2.000010e+12,9,UN,Unico,PZ,UN,BS,BUONO SCONTO,UN,AI,...,1,5.0,0.01,0,,,0,NC,NC,NC
1,2.000000e+12,1,UN,Unico,PZ,UN,GF,GIFT CARD,UN,AI,...,1,15.0,5.00,-7,,,0,NC,NC,NC
2,2.000000e+12,2,UN,Unico,PZ,UN,GF,GIFT CARD,UN,AI,...,1,25.0,8.33,-13,,,0,NC,NC,NC
3,2.000000e+12,2,UN,Unico,PZ,UN,GF,GIFT CARD,UN,AI,...,1,0.0,0.00,0,,,0,NC,NC,NC
4,2.000000e+12,3,UN,Unico,PZ,UN,GF,GIFT CARD,UN,AI,...,1,50.0,16.66,-1,,,0,NC,NC,NC


In [5]:
item_cluster = items.drop(['Barcode', 'Color_code'],1)

item_cluster.head()

,Product_identifier,Color_description,Size_code,Gender,Item,Item_description,Product_category_code,season_code,year,colour_ID,size_ID,Original_list_price,COG,Recieved_items,Introduction_period,rotation,price_range,fashion_content,seasonality,innovation_content
0,9,Unico,PZ,UN,BS,BUONO SCONTO,UN,AI,1990,1,1,5.0,0.01,0,,,0,NC,NC,NC
1,1,Unico,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,1,15.0,5.00,-7,,,0,NC,NC,NC
2,2,Unico,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,1,25.0,8.33,-13,,,0,NC,NC,NC
3,2,Unico,PZ,UN,GF,GIFT CARD,UN,AI,1990,2,1,0.0,0.00,0,,,0,NC,NC,NC
4,3,Unico,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,1,50.0,16.66,-1,,,0,NC,NC,NC


In [6]:
print(item_cluster.shape)

cluster_dropna = item_cluster.dropna()

print(cluster_dropna.shape)

cluster_dropna = cluster_dropna.astype({"Gender": str, "price_range": str})


(448818, 20)
(428885, 20)


In [7]:
#just try to assert that all str should be int. Look for a pandas function for this!

i = 0
for x in item_cluster['price_range']:
    if type(x) == str:
#         print(x)
        i += 1

print(i)

55602


In [8]:
#so not available gender isn't properly coded here - check if this corresponds to other NaNs that you removed! the like 40k of them. 

i = 0
for x in item_cluster['Gender']:
    if type(x) == float:
        
        i += 1

print(i)

19933


In [19]:
categorical_variables = [] 
category_names = []
for i,x in enumerate(cluster_dropna.columns):
    type_set = set()
    print(i,x, len(set(cluster_dropna[x])), type(list(cluster_dropna[x])[0]))
    for y in cluster_dropna[x]:
        type_set.add(type(y))
    print(type_set)
    if type(list(cluster_dropna[x])[0]) == str:
        categorical_variables.append(i)
        category_names.append(x)
    if type(list(item_cluster[x])[0]) == int:
        categorical_variables.append(i)
        category_names.append(x)

0 Product_identifier 40719 <class 'int'>
{<class 'int'>}
1 Color_description 416 <class 'str'>
{<class 'str'>}
2 Size_code 82 <class 'str'>
{<class 'str'>}
3 Gender 7 <class 'str'>
{<class 'str'>}
4 Item 130 <class 'str'>
{<class 'str'>}
5 Item_description 129 <class 'str'>
{<class 'str'>}
6 Product_category_code 8 <class 'str'>
{<class 'str'>}
7 season_code 3 <class 'str'>
{<class 'str'>}
8 year 9 <class 'int'>
{<class 'int'>}
9 colour_ID 27 <class 'int'>
{<class 'int'>}
10 size_ID 15 <class 'int'>
{<class 'int'>}
11 Original_list_price 142 <class 'float'>
{<class 'float'>}
12 COG 1460 <class 'float'>
{<class 'float'>}
13 Recieved_items 7212 <class 'int'>
{<class 'int'>}
14 Introduction_period 9 <class 'str'>
{<class 'str'>}
15 rotation 8 <class 'str'>
{<class 'str'>}
16 price_range 123 <class 'str'>
{<class 'str'>}
17 fashion_content 4 <class 'str'>
{<class 'str'>}
18 seasonality 4 <class 'str'>
{<class 'str'>}
19 innovation_content 5 <class 'str'>
{<class 'str'>}


In [20]:
print(category_names)

['Product_identifier', 'Color_description', 'Size_code', 'Gender', 'Item', 'Item_description', 'Product_category_code', 'season_code', 'year', 'colour_ID', 'size_ID', 'Recieved_items', 'Introduction_period', 'rotation', 'price_range', 'price_range', 'fashion_content', 'seasonality', 'innovation_content']


In [ ]:
#scale all between -1 and 1?
#PCA of some BERT fashion vectors for Item and Item description?
#RGB colors from https://htmlcolorcodes.com/color-names/?
#-1 and 1 for gender


In [21]:
categorical_frame = cluster_dropna.filter(items=category_names, axis=1)

In [22]:
print(categorical_frame.head())

   Product_identifier Color_description Size_code Gender Item  \
0                   9             Unico        PZ     UN   BS   
1                   1             Unico        PZ     UN   GF   
2                   2             Unico        PZ     UN   GF   
3                   2             Unico        PZ     UN   GF   
4                   3             Unico        PZ     UN   GF   

  Item_description Product_category_code season_code  year  colour_ID  \
0     BUONO SCONTO                    UN          AI  1990          1   
1        GIFT CARD                    UN          AI  1990          1   
2        GIFT CARD                    UN          AI  1990          1   
3        GIFT CARD                    UN          AI  1990          2   
4        GIFT CARD                    UN          AI  1990          1   

   size_ID  Recieved_items Introduction_period rotation price_range  \
0        1               0                                        0   
1        1              -7  

In [24]:
from kmodes.kmodes import KModes

km = KModes(n_clusters=4, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(categorical_frame)

print(km.cluster_centroids_)


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 99144, cost: 4748395.0
Run 1, iteration: 2/100, moves: 13845, cost: 4748395.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 161302, cost: 4761951.0
Run 2, iteration: 2/100, moves: 3771, cost: 4761951.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 140219, cost: 4686488.0
Run 3, iteration: 2/100, moves: 9, cost: 4686488.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 164447, cost: 4677570.0
Run 4, iteration: 2/100, moves: 11787, cost: 4677570.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 105060, cost: 4693807.0
Run 5, iteration: 2/100, moves: 27503, cost: 4693807.0
Best run was number 4


array([0, 0, 0, ..., 1, 3, 1], dtype=uint16)

In [ ]:

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 16, 17, 18, 19]

In [23]:
from kmodes.kprototypes import KPrototypes 

K=10

KP = KPrototypes(n_clusters=2, init='Cao', verbose=1)

KP.fit(cluster_dropna, categorical = categorical_variables)

# KP.fit_predict(cluster_dropna, categorical = categorical_variables)


# oh hey look another bug. This time it's with 

Init: initializing centroids
Init: initializing clusters


IndexError: index 1 is out of bounds for axis 1 with size 1

AttributeError: '{}' object has no attribute 'cluster_centroids_' because the model is not yet fitted.

In [4]:
import pandas as pd
import numpy as np
import time
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import random


from sklearn import cluster
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS, Isomap
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")



In [4]:
from kmodes.kmodes import KModes

In [ ]:
#Build store cluster model - these can be your categories. Categories for what?

#you could try to identify outliers in these clusters - is there a store that is underperforming in a cluster? if so, 
#in what ways does it differ in terms of allocation of goods etc?



#https://pypi.org/project/pgeocode/ - can convert italy ('it') zip to lat & long







In [ ]:
#Build item cluster model that is relatively well-cleaned. 

#https://towardsdatascience.com/clustering-on-mixed-type-data-8bbd0a2569c3



# If your data contains both numeric and categorical variables, the best way to carry out clustering on the dataset is to create principal components of the dataset and use the principal component scores as input into the clustering.

# Remember that u can always get principal components for categorical variables using a multiple correspondence analysis (MCA), which will give principal components, and you can get then do a separate PCA for the numerical variables, and use the combined as input into your clustering.

# OR u could use the R package called FactorMineR or PCAmix to carry Factor analysis of mixed data, with the output being principal components, and then using the principal components as input into your clustering.

# Remember that clustering and principal components are doing almost similar thing in a simplistic way they're both distances, e.g Euclidean distance.

#https://datascience.stackexchange.com/questions/22/k-means-clustering-for-mixed-numeric-and-categorical-data - K-modes approach to clustering.

# you could also use a K-modes approach to clustering, which might have some value - https://medium.com/@davidmasse8/unsupervised-learning-for-categorical-data-dd7e497033ae